# Chapter One - Data Processing with Optimus 

## Hi Optimus! 

### One API to rule them all 

Some examples of how datasets are created without using Optimus

#### pandas

In [ ]:
import pandas as pd 
import numpy as np 

data = {"A":[0, 1,2,3,4,5]} 
pdf = pd.DataFrame(data) 
pdf["A"] = np.sqrt(pdf["A"]) 
pdf 

#### cuDF

In [ ]:
try:
    import cudf

    # You need to use dtype=”float” to return floats 

    cdf = cudf.DataFrame(data, dtype = "float") 
    cdf["A"] = cdf["A"].sqrt() 
except:
    cdf = "This could not run in your machine if "\
          "you don't have cuDF or any compatible "\
          "drivers installed."
cdf

#### dask

In [ ]:
from dask import dataframe as dd  
# You cannot create a dataframe directly from a dict. You need to use a pandas dataframe. 
ddf = dd.from_pandas(pdf, npartitions=1) 

import dask.array as da 
ddf["A"]=da.sqrt(ddf.A) 

# You need to user compute to materialize the pandas dataframe 
ddf.compute() 

#### dask cuDF

In [ ]:
try:
    import dask_cudf 

    dcdf = dask_cudf.from_cudf(cdf, npartitions=2) 
    dcdf.map_partitions(cudf.sqrt).compute()
except:
    dcdf = "This could not run in your machine if "\
           "you don't have Dask-cuDF or any compatible "\
           "drivers installed."
dcdf

#### vaex

In [ ]:
import vaex 
import numpy as np 
  
data = {"A":[0, 1,2,3,4,5]}
vdf = vaex.from_dict(data)
vdf["A"] = np.sqrt(vdf["A"]) 
vdf 

## Using Optimus 

Instantiating Optimus

In [ ]:
from optimus import Optimus 
op = Optimus("pandas")

Using a Dask remote cluster

In [ ]:
from dask.distributed import Client 
client = Client("127.0.0.105") 
op = Optimus(engine="dask", session=client) 

Using Dask locally with a different number of workers

In [ ]:
op = Optimus(engine="dask", n_workers=2)

Loading a file

In [ ]:
op.load.csv("path/to/file.csv")

In [ ]:
df = op.load.file("path/to/file.csv")
df

Connecting to a database or an external bucket and loading a file

In [ ]:
db = op.connect.database(*db_args)
op.load.database_table("table name", connection=db) 

In [ ]:
conn = op.connect.s3(*s3_args) 
op.load.file("relative/path/to/file.csv", connection=conn) 

Saving a file

In [ ]:
df.save.csv("path/to/output-file.csv") 

In [ ]:
df.save.database_table("table_name", db=db) 

In [ ]:
df.save.csv("relative/path/to/output-file.csv", connection=conn) 

## The Optimus Dataframe

In [ ]:
df.cols.rename("function", "job") 

In [ ]:
df = df.cols.rename("function", "job")
df

In [ ]:
df = df.cols.upper("name").cols.lower("job")
df

In [ ]:
df.cols.drop("name") 

In [ ]:
df.rows.drop(df["name"]=="MEGATRON") 

In [ ]:
df.display()

In [ ]:
dfn = op.create.dataframe({"A":["1",2,"4","!",None]})

In [ ]:
dfn.cols.min("A"), dfn.cols.max("A")

In [ ]:
df.cols.capitalize("name", output_cols="cap_name") 

In [ ]:
df = op.create.dataframe({
    "A":["1",2,"4","!",None],
    "B":["Optimus","Bumblebee", "Eject", None, None]
})  
df.profile(bins=10) 

In [ ]:
df.columns_sample("*") 

In [ ]:
df.execute()

In [ ]:
op = Optimus("pandas") 
df = op.load.csv("foo.txt", sep=",") 
type(df.data)

## Meta

In [ ]:
from optimus import Optimus  
op = Optimus("pandas")  
df = op.load.csv("foo.txt", sep=",") 
df.meta 

### Actions

In [ ]:
from optimus import Optimus  
op = Optimus("pandas")  
df = op.load.csv("foo.txt", sep=",") 
df = df.cols.upper("*") 

In [ ]:
df.meta["transformations"] 